# Ensambles
## La idea es poder lograr algo del siguiente estilo:

pred_1 = modelo_1(df, df_test)

pred_2 = modelo_2(df, df_test)

pred_3 = modelo_3(df, df_test)

pred_4 = modelo_4(df, df_test)

## Con esto se puede evaluar y sacar un promedio entre las predicciones o hacer otros tipos de ensambles

In [1]:
from models import pred_with_rf, pred_with_xgboost
import pandas as pd

In [2]:
df = pd.read_csv('data/train.csv',
        index_col='id',
        dtype={'gimnasio': int,
                'usosmultiples': int,
                'escuelascercanas': int,
                'piscina': int,
                'centroscomercialescercanos': int,
                'tipodepropiedad': 'category',
                'provincia': 'category',
                'ciudad': 'category'
            },
        parse_dates=['fecha'])

df_test = pd.read_csv('data/test.csv',
        index_col='id',
        dtype={'gimnasio': int,
                'usosmultiples': int,
                'escuelascercanas': int,
                'piscina': int,
                'centroscomercialescercanos': int,
                'tipodepropiedad': 'category',
                'provincia': 'category',
                'ciudad': 'category'
            },
        parse_dates=['fecha'])


In [3]:
pred_2 = pred_with_xgboost(df, df_test)

/home/ggparente95/Documentos/Facu/Datos/TP2/tp2-zonajobs/XGBoostRam.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[idsNullMetrosTotales]['metrostotales'] =  df[idsNullMetrosTotales]['metroscubiertos']
/home/ggparente95/Documentos/Facu/Datos/TP2/tp2-zonajobs/XGBoostRam.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[idsNullMetrosCubiertos]['metroscubiertos'] =  df[idsNullMetrosCubiertos]['metrostotales']



Despues de filtrar:  (223475, 20)


/home/ggparente95/.local/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/home/ggparente95/.local/lib/python3.7/site-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


In [6]:
pred_1 = pred_with_rf(df, df_test)

In [7]:
final_pred = (pred_1+pred_2)/2

# Submit
# Score: 573k

In [8]:
res = pd.DataFrame(final_pred, index=df_test.index, columns=['target'])
display(res.head())
res.to_csv("data/submission-rfxg.csv", header=True)

,target
id,
4941,4.706227e+06
51775,9.876805e+05
115253,2.586411e+06
299321,1.533391e+06
173570,5.825126e+05
